In [4]:
from azureml.core.run import Run
from azureml.core import Dataset, Datastore, Workspace
from azureml.core.runconfig import RunConfiguration
from azureml.core import Environment, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.datadrift import DataDriftDetector
from datetime import datetime
import calendar


import pandas as pd
from azureml.core import Workspace
ws = Workspace.from_config()


# repeat for pv, npv and bs
veg_type = "npv"


# register dataset from datastore
blob_datastore = Datastore.get(ws, datastore_name = "vdsmtrain")

# create a TabularDataset from file in datastore
blob_path = "data_drift/target/*/" +  veg_type + ".csv"
partition_format = 'data_drift/target/{ymd-drift:yyyy-MM}/'  + veg_type +'.csv'

datastore_paths = [(blob_datastore, blob_path)]
target_data_set = Dataset.Tabular.from_delimited_files(path=datastore_paths, partition_format=partition_format)

# assign the timestamp attribute to a real or virtual column in the dataset
target_data = target_data_set.with_timestamp_columns('ymd-drift')


# extract the ymd-drift
ymd_drift_col = target_data.to_pandas_dataframe()['ymd-drift']
yyyy_min_drift = min(ymd_drift_col).year
mm_min_drift = min(ymd_drift_col).month
yyyy_max_drift = max(ymd_drift_col).year
mm_max_drift = max(ymd_drift_col).month


# register the dataset as the target dataset
registered_data_name = 'monthly drift_target_' + veg_type
target_dataset = target_data.register(workspace=ws, 
                           name=registered_data_name,
                           description='monthly drift target for ' + veg_type,
                           tags = {'format':'CSV', 'category':veg_type},
                           create_new_version=True)


cluster_name = "cluster-drift"

# set up feature list
features = ['evi']

registered_baseline_data_name = 'drift baseline_' + veg_type
baseline_dataset = Dataset.get_by_name(ws, name= registered_baseline_data_name )


# set up data drift detector
data_drift_name = veg_type + '-mdrift-' 

monitor = DataDriftDetector.create_from_datasets(ws, data_drift_name, 
                                                       baseline_dataset, 
                                                       target_dataset,
                                                      compute_target=cluster_name, 
                                                      frequency='Month', 
                                                      feature_list=features, 
                                                      drift_threshold=.3, 
                                                      latency=0)    